<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/vgg19_6classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from timeit import default_timer as timer

start = timer()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

from random import randint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras import layers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE = 32
TEST_BATCH_SIZE=32
TEST_STEPS=1
IMAGE_SIZE = (224, 224)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/dogeye.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_d')

In [ ]:
base_dir = '/content/unzipp_d/dogeye'

train_dir = '/content/unzipp_d/dogeye/train'
cat_train_dir = '/content/unzipp_d/dogeye/train/cataract'
healthy_train_dir = '/content/unzipp_d/dogeye/train/healthy'
mas_train_dir = '/content/unzipp_d/dogeye/train/mas'
conj_train_dir = '/content/unzipp_d/dogeye/train/conj'
ulcer_train_dir = '/content/unzipp_d/dogeye/train/ulcer'
nuc_train_dir = '/content/unzipp_d/dogeye/train/nuc'

test_dir = '/content/unzipp_d/dogeye/test'
cat_test_dir = '/content/unzipp_d/dogeye/test/cataract'
healthy_test_dir = '/content/unzipp_d/dogeye/test/healthy'
mas_test_dir = '/content/unzipp_d/dogeye/test/mas'
conj_test_dir = '/content/unzipp_d/dogeye/test/conj'
ulcer_test_dir = '/content/unzipp_d/dogeye/test/ulcer'
nuc_test_dir = '/content/unzipp_d/dogeye/test/nuc'

val_dir = '/content/unzipp_d/dogeye/valid'
cat_val_dir = '/content/unzipp_d/dogeye/valid/cataract'
healthy_val_dir = '/content/unzipp_d/dogeye/valid/healthy'
mas_val_dir = '/content/unzipp_d/dogeye/valid/mas'
conj_val_dir = '/content/unzipp_d/dogeye/valid/conj'
ulcer_val_dir = '/content/unzipp_d/dogeye/valid/ucler'
nuc_val_dir = '/content/unzipp_d/dogeye/valid/nuc'

In [ ]:
import os
#train
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))

num_mas_train = len(os.listdir(mas_train_dir))
num_conj_train = len(os.listdir(conj_train_dir))
num_ulcer_train = len(os.listdir(ulcer_train_dir))
num_nuc_train = len(os.listdir(nuc_train_dir))

#test
num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))

num_mas_test = len(os.listdir(mas_test_dir))
num_conj_test = len(os.listdir(conj_test_dir))
num_ulcer_test = len(os.listdir(ulcer_test_dir))
num_nuc_test = len(os.listdir(nuc_test_dir))

#val
num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))

num_mas_val = len(os.listdir(mas_val_dir))
num_conj_val = len(os.listdir(conj_val_dir))
num_ulcer_val = len(os.listdir(ulcer_val_dir))
num_nuc_val = len(os.listdir(nuc_val_dir))

In [ ]:
print("--")
print("Total Training cat Images",num_cat_train)
print("Total Training conj Images",num_conj_train)

print("Total Training healthy Images",num_healthy_train)
print("Total Training mas Images",num_mas_train)
print("Total Training nuc Images",num_nuc_train)
print("Total Training ulcer Images",num_ulcer_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)

print("Total test mas Images",num_mas_test)
print("Total Training conj Images",num_conj_test)
print("Total Training ulcer Images",num_ulcer_test)
print("Total Training nuc Images",num_nuc_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)

print("Total validation mas Images",num_mas_val)
print("Total Training conj Images",num_conj_val)
print("Total Training ulcer Images",num_ulcer_val)
print("Total Training nuc Images",num_nuc_val)
print("--")

In [ ]:
total_train = num_cat_train+num_healthy_train+num_mas_train+num_conj_train+num_ulcer_train+num_nuc_train
total_validation = num_cat_val+num_healthy_val+num_mas_val+num_conj_val+num_ulcer_val+num_nuc_val
total_test = num_cat_test+num_healthy_test+num_mas_test+num_conj_test+num_ulcer_test+num_nuc_test

total_images = total_train + total_validation + total_test

print("--")
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)
print("--")
print("Total Images",total_images)
print("--")

In [ ]:
print ( 'test batch size: ' ,TEST_BATCH_SIZE, '  test steps: ', TEST_STEPS)

In [ ]:
import numpy as np

def scalar(img):
    return img  # EfficientNet expects pixels in range 0 to 255 so no scaling is required
trgen=ImageDataGenerator(preprocessing_function=scalar, horizontal_flip=True)
tvgen=ImageDataGenerator(preprocessing_function=scalar)
train_generator=trgen.flow_from_directory( directory=train_dir , target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=BATCH_SIZE)
test_generator=tvgen.flow_from_directory( directory=test_dir, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=False, batch_size=TEST_BATCH_SIZE)

valid_generator=tvgen.flow_from_directory( directory=val_dir, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=BATCH_SIZE)
classes=list(train_generator.class_indices.keys())
class_count=len(classes)
train_steps=int(np.ceil(len(train_generator.labels)/BATCH_SIZE))

In [ ]:
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5)

In [ ]:
from keras.applications.vgg19 import VGG19
base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (224,224,3), classes = 6)

In [ ]:
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

model= Sequential()
model.add(base_model)
model.add(Flatten())

In [ ]:
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu')))
model.add(Dense(256,activation=('relu')))
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
model.add(Dense(6,activation=('softmax'))) # 7 for seven classes

#Checking the final model summary
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.optimizers import SGD,Adam
epochs=5
learn_rate=.001
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam = Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# model.fit(train_generator, epochs=epochs, validation_data=valid_generator, verbose=1, callbacks=[early_stopping])


In [ ]:
model.fit(train_generator,epochs = epochs, validation_data = valid_generator, verbose = 1)

In [ ]:
model.evaluate(test_generator)

In [ ]:
import numpy as np
import sklearn.metrics as metrics

y_pred = model.predict(test_generator)

y_pred = np.argmax(y_pred, axis=1)


y_true = test_generator.classes

report = metrics.classification_report(y_true, y_pred, target_names=classes)
print(report)


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
print(confusion_matrix)

In [ ]:
end = timer()
print(end - start)